# Simple Facial Recognition Using Dlib And OpenCv
The facial recognition system is a computer vision application that can detect and recognize faces in images or video. In this system, dlib and OpenCV libraries are used for modeling and image processing tasks respectively.
## Dlib

Dlibis a modern C++ toolkit containing machine learning algorithms specilly computer vision and tools for creating complex software in C++ and python to solve real world problems. It is used in both industry and academia. 

[Dlib Official Sites](http://dlib.net/)

## OpenCV 
OpenCV (Open Source Computer Vision Library) is an open source computer vision and machine learning software library. OpenCV was built to provide a common infrastructure for computer vision applications and to accelerate the use of machine perception in the commercial products.

[OpenCv Official Sites](https://opencv.org/about/)

### Imports

In [1]:
from imutils import paths
import face_recognition
import imutils
import pickle
import cv2
import os
import time

## Generating Face Encodings with Dlib
- To find face bounding boxes with dlib, we use the face_locations() function from the face_recognition library.     This function takes an image as input and returns the bounding boxes of all the faces present in the image.
- The face_locations() function uses a dlib model (by default, the HOG model) to detect faces in the image.
- To compute the facial embeddings or encodings with dlib, we use the face_encodings() function from the             face_recognition library. This function takes an image and a list of face bounding boxes as input, and returns a   list of 128-dimensional vectors representing the facial features of each face in the image.
- The resulting encodings can be used for face recognition.

In [2]:
imagePaths = list(paths.list_images('training-dataset'))
knownEncodings = []
knownNames = []

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    name = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

## Face Recognition Using Dlib 
- Detecting the faces using the Haar cascades and storing the bounding box coordinates.
- Generating the facial encodings for the detected faces using dlib face_encodings function.
- Loading the known faces and encodings saved from system which we created.
- Comparing the new generated user encodings with the  encodings in the data set using dlib compare_faces           function.
- Drawing a rectangle around the detected face and displaying the predicted name on the output image than save and   show the ouput image 

In [3]:
#find path of xml file containing haarcascade file
cascPathface = os.path.dirname(
 cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())
# Input image path
input_image_path = 'input-images/2.jpeg'
image_name = input_image_path.split('/')
output_image_path = 'output-images/predicted-'+image_name[1]
image = cv2.imread(input_image_path)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#convert image to Greyscale for haarcascade
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(60, 60),flags=cv2.CASCADE_SCALE_IMAGE)
 
# the facial embeddings for face in input
encodings = face_recognition.face_encodings(rgb)
names = []
# loop over the facial embeddings incase
# we have multiple embeddings for multiple fcaes
for encoding in encodings:
    #Compare encodings with encodings in data["encodings"]
    #Matches contain array with boolean values and True for the embeddings it matches closely
    #and False for rest
    matches = face_recognition.compare_faces(data["encodings"],
    encoding)
    name = "Unknown"
    # check to see if we have found a match
    if True in matches:
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            #Check the names at respective indexes we stored in matchedIdxs
            name = data["names"][i]
            #increase count for the name we got
            counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
        names.append(name)
        print('Model Predicted :',names[0])
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            cv2.rectangle(image, (x, y), (x + w, y + h), (0,0, 255), 8)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 4)
    else :
        names.append(name)
        print("Model Predicted : ",names[0])

        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            cv2.rectangle(image, (x, y), (x + w, y + h), (0,0, 255), 8)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 4)

    cv2.imwrite(output_image_path+'-'+names[0]+'.png',image)
    cv2.imshow(names[0],image)    
    cv2.waitKey(0)

Model Predicted : Steve Jobs


## Final Thoughts 
This simple facial recognition system uses the dlib library to detect faces in an input image, generates face encodings using pre-trained models, and matches these encodings with a pre-saved dataset of known faces to predict the identity of the input image. For more advanced and accurate facial recognition systems, one can train machine learning models such as KNN or CNN on large datasets to improve the accuracy of facial recognition.

The CNN or KNN algorithm can achieve higher accuracy than the simple facial recognition system using dlib because it can learn more complex patterns and relationships between the face embeddings, and it can generalize better to new faces. However, it requires more computational resources and training data to achieve good performance.